In [4]:
# important paths
DATA_PATH = r"..\data\processed\processed_1.pkl"
FEATURES_PATH = r"..\data\processed\cleaned_df.pkl"


In [5]:
# used libraries
import pandas as pd
import mlflow
import os

import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import warnings

In [6]:
warnings.filterwarnings("ignore")

In [7]:
# reading data
processed_df = pd.read_pickle(DATA_PATH)
combined_df = pd.read_pickle(FEATURES_PATH)
df = combined_df.copy()

In [9]:
roles_df = combined_df['DevType'].copy()
roles_df.sum(axis=0)

Academic researcher                               1283
Cloud infrastructure engineer                      931
Data or business analyst                           813
Data scientist or machine learning specialist     1474
Database administrator                             245
DevOps specialist                                 1217
Developer, QA or test                              562
Developer, back-end                              13045
Developer, desktop or enterprise applications     3745
Developer, embedded applications or devices       1780
Developer, front-end                              4915
Developer, full-stack                            23303
Developer, game or graphics                        824
Developer, mobile                                 2512
Engineer, data                                    1170
Hardware Engineer                                  278
Scientist                                          331
Security professional                              416
System adm

#### Handling the imbalance

In [10]:
# Resample roles
samples_per_class = 1400
resampled_roles = []

In [11]:
for role_col in roles_df.columns:
    sub_df = roles_df.loc[roles_df[role_col] == 1].copy()

    if len(sub_df) / samples_per_class < 1 :
        # Upsample
        t = len(sub_df) *   (len(sub_df) / samples_per_class) 
        amo = t + samples_per_class
        sub_df = sub_df.sample(int(amo), replace=True, random_state=0)
    else:
        # Downsample
        t = (len(sub_df) / samples_per_class) 
        if t > 2 :
            t/= 2
        amo = samples_per_class * t

        sub_df = sub_df.sample(int(amo), random_state=0)

    resampled_roles.append(sub_df)

In [12]:
# Construct dfs
roles_df = pd.concat(resampled_roles)
df = df.loc[roles_df.index].copy()

In [13]:
roles_df.sum(axis=0)

Academic researcher                               2575
Cloud infrastructure engineer                     2019
Data or business analyst                          1872
Data scientist or machine learning specialist     1474
Database administrator                            1442
DevOps specialist                                 2457
Developer, QA or test                             1625
Developer, back-end                               6522
Developer, desktop or enterprise applications     1872
Developer, embedded applications or devices       1780
Developer, front-end                              2457
Developer, full-stack                            11651
Developer, game or graphics                       1884
Developer, mobile                                 2512
Engineer, data                                    2377
Hardware Engineer                                 1455
Scientist                                         1478
Security professional                             1523
System adm

In [14]:
X=df['Clusters']
y = df['DevType'].idxmax(axis=1)

### Basic RandomForest model

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15,stratify=y,  random_state=42)

In [16]:
numeric_cols = X_train.columns

In [17]:
preprocessor = ColumnTransformer(
                transformers=[
                    ('num', RobustScaler(), numeric_cols)
                ]
)

In [42]:
rf_clf = Pipeline([
    
    ('preprocessor', preprocessor),
    ('pca', PCA(n_components=0.99)),
    ('classifier', RandomForestClassifier(n_jobs=-1, verbose=1, random_state=42))
])

rf_clf.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    1.6s finished


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', RobustScaler(),
                                                  Index(['skills_group_0', 'skills_group_1', 'skills_group_10',
       'skills_group_11', 'skills_group_12', 'skills_group_13',
       'skills_group_14', 'skills_group_15', 'skills_group_16',
       'skills_group_17', 'skills_group_18', 'skills_group_19',
       'skills_group_2', 'skills_group_20', 'skills_group_21',
       'skills_group_22', 'skills_group_23', 'skills_group_24',
       'skills_group_3', 'skills_group_4', 'skills_group_5', 'skills_group_6',
       'skills_group_7', 'skills_group_9'],
      dtype='object'))])),
                ('pca', PCA(n_components=0.99)),
                ('classifier',
                 RandomForestClassifier(n_jobs=-1, random_state=42,
                                        verbose=1))])

In [43]:
y_pred = rf_clf.predict(X_test)

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished


In [44]:
print (classification_report(y_test, y_pred))

                                               precision    recall  f1-score   support

                          Academic researcher       0.77      0.78      0.78       386
                Cloud infrastructure engineer       0.91      0.88      0.89       303
                     Data or business analyst       0.81      0.81      0.81       281
Data scientist or machine learning specialist       0.68      0.54      0.60       221
                       Database administrator       0.88      0.94      0.91       216
                            DevOps specialist       0.91      0.81      0.86       369
                        Developer, QA or test       0.91      0.83      0.87       244
                          Developer, back-end       0.52      0.39      0.45       978
Developer, desktop or enterprise applications       0.35      0.16      0.22       281
  Developer, embedded applications or devices       0.45      0.43      0.44       267
                         Developer, front-

In [53]:
f1=f1_score(y_test, y_pred, average='weighted')
f1

0.6828606760279509

In [49]:
precision=precision_score(y_test, y_pred, average='weighted')

In [50]:
recall=recall_score(y_test, y_pred, average='weighted')

In [46]:
# where to create the mlruns folder
MLFLOW_TRACKING_URI = '../models/mlruns'
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)


In [47]:
experiment_name = "rf_job_predict"
artifact_location = r"..\models\Models"
experiment_id = mlflow.create_experiment(experiment_name, artifact_location)


In [48]:
# Set MLFLOW_EXPERIMENT_LOCATION to the desired artifact location
os.environ["MLFLOW_EXPERIMENT_LOCATION"] = artifact_location

In [55]:
with mlflow.start_run(experiment_id=experiment_id):
    # Log parameters
    mlflow.log_param("PCA n_components", 0.99)

    # Log metrics
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("precision", precision)
    mlflow.sklearn.log_model(rf_clf, " Basic RandomForestClassifier with PCA")